In [1]:
import torch
import os
from PIL import Image
import torchvision.transforms as T
import pandas as pd
from rle import rle_decode
import numpy as np
from surface_dice import compute_surface_dice_score


train_dir = "/kaggle/input/blood-vessel-segmentation/train/"
msks_dir = f"{train_dir}kidney_1_dense/labels/"
imgs_dir = f"{train_dir}kidney_1_dense/images/"
slices_ids = sorted(os.listdir(imgs_dir))
device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
w = Image.open(msks_dir + slices_ids[0]).width
h = Image.open(msks_dir + slices_ids[0]).height
print(w, h)

912 1303


In [3]:
class TestMetricDataset(torch.utils.data.Dataset):
    def __init__(self, sub_df, msks_dir, slices_ids, transform=None, target_transform=None):
        self.sub_df = sub_df
        self.msks_dir = msks_dir
        self.slices_ids = slices_ids
        self.transform = transform
        self.target_transform = target_transform
        
    def __len__(self):
        return len(self.slices_ids)
    
    def __getitem__(self, idx):
        slice_id = self.slices_ids[idx]

        pred_rle = sub_df.iloc[idx]["rle"]
        pred = rle_decode(pred_rle, (h, w))
        pred = torch.from_numpy(pred)

        target_path = self.msks_dir + slice_id 
        target = Image.open(target_path)

        if self.target_transform is not None:
            target = self.target_transform(target).squeeze()

        return pred, target

In [4]:
sub_df = pd.read_csv("ref_sub.csv")

target_transform = T.Compose([
    T.PILToTensor(), 
])

ds = TestMetricDataset(sub_df=sub_df, msks_dir=msks_dir, slices_ids=slices_ids, target_transform=target_transform)
dl = torch.utils.data.DataLoader(ds, batch_size=5)

In [5]:
from tqdm import tqdm
n, d = 0, 0

for pred, target in tqdm(dl):
    pred, target = pred.to(device), target.to(device)
    # print(pred.shape)
    # print(target.shape)
    bn, bd = compute_surface_dice_score(pred, target, device=device, return_num_denom=True)
    n += bn
    d += bd

dice = n / d.clamp(min=1e-8)
dice = dice.item()
dice

  2%|▏         | 10/456 [00:01<00:53,  8.35it/s]/usr/local/src/pytorch/aten/src/ATen/native/cuda/IndexKernel.cu:92: operator(): block: [903,0,0], thread: [41,0,0] Assertion `index >= -sizes[i] && index < sizes[i] && "index out of bounds"` failed.
/usr/local/src/pytorch/aten/src/ATen/native/cuda/IndexKernel.cu:92: operator(): block: [903,0,0], thread: [42,0,0] Assertion `index >= -sizes[i] && index < sizes[i] && "index out of bounds"` failed.
/usr/local/src/pytorch/aten/src/ATen/native/cuda/IndexKernel.cu:92: operator(): block: [906,0,0], thread: [75,0,0] Assertion `index >= -sizes[i] && index < sizes[i] && "index out of bounds"` failed.
/usr/local/src/pytorch/aten/src/ATen/native/cuda/IndexKernel.cu:92: operator(): block: [906,0,0], thread: [76,0,0] Assertion `index >= -sizes[i] && index < sizes[i] && "index out of bounds"` failed.
/usr/local/src/pytorch/aten/src/ATen/native/cuda/IndexKernel.cu:92: operator(): block: [904,0,0], thread: [58,0,0] Assertion `index >= -sizes[i] && index < 

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
